In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os
import numpy as np
import sys

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [ ]:
##  configuration options
config_file = '/home/cosmonaut/dev/PIC/pdp3/td/parameters.xml'

cfg = Parameters(config_file)

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
shape=[0,0,cfg.number_r_grid, cfg.number_z_grid]

timestamp=1e-10
show_grid=False
use_cache=False
cmap='terrain'

# color limits (WARNING: clim_estimation may works incorrectly)
clim_t = [0, 4]
specie = 'electrons'

image_interpolation = 'nearest'

use_grid = False
ylim = None
autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\mathit{T (eV)}$'

plot_name = r'$\mathbf{Temperature\enspace Map\enspace (T)}$'

In [ ]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         data_root=cfg.data_root,
                         fullframe_size=[cfg.number_r_grid , cfg.number_z_grid],
                         fpds=cfg.frames_per_file, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

r_scale = (shape[2] - shape[0]) / cfg.number_r_grid
z_scale = (shape[3] - shape[1]) / cfg.number_z_grid

In [ ]:
# get data
data = []

for probe in cfg.probes:
    frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.type == 'frame') and (probe.specie == specie) and (probe.r_start == shape[0]) and (probe.z_start == shape[1]) and(probe.r_end == shape[2]) and(probe.z_end == shape[3]):
        if probe.component == 'T': data = reader.get_frame('T/{}'.format(probe.specie), shape, frame)

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        probe_shape = [probe.r_start, probe.z_start, probe.r_end, probe.z_end]
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.type == 'frame') and (probe.specie == specie) and (probe_shape[0] <= shape[0]) and (probe_shape[1] <= shape[1]) and(probe_shape[2] >= shape[2]) and(probe_shape[3] >= shape[3]):
            if probe.component == 'T' and len(data) == 0: data = reader.get_frame('T/{}'.format(probe.specie), probe_shape, frame)[shape[0]:shape[2], shape[1]:shape[3]]

In [ ]:
data_flat = data.flatten()

In [ ]:
min_ = min(data_flat)
max_ = max(data_flat)
sampling_amount = 100

data_to_plot = np.zeros(100)

sampling_values = np.linspace(min_, max_, sampling_amount)

In [ ]:
for i in data_flat:
    for j in range(0, sampling_amount):
        if i > sampling_values[j] and i <= sampling_values[j+1]:
            data_to_plot[j] += 1

In [ ]:
# define plot builder
plot = PlotBuilder(0, 0, # let the system detects sizes automatically
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20,
                   # number_x_ticks=10, number_y_ticks=10
                   x_ticklabel_end=1e-9, y_ticklabel_end=1e-9
                  )

# add subplots
plot_obj = plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# set y-limits
if ylim is not None:
    plot_obj.set_ylim(ylim)

In [ ]:
# add timestamp to plot
# plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot_obj.plot(sampling_values, data_to_plot)

plot.redraw()